## Loss Functions

In this assignment, we will learn about loss functions. We will use a create a neural network and measure the model's performance using different loss functions.

In [15]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 100)

In [2]:
housing = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/housing.csv')

In [3]:
housing.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


We will use the dataset above to predict housing prices using various features about each house. Our first step is to check for missing data. 

In [4]:
# Answer below:
housing.isna().mean()

Id               0.000000
MSSubClass       0.000000
MSZoning         0.000000
LotFrontage      0.177397
LotArea          0.000000
Street           0.000000
Alley            0.937671
LotShape         0.000000
LandContour      0.000000
Utilities        0.000000
LotConfig        0.000000
LandSlope        0.000000
Neighborhood     0.000000
Condition1       0.000000
Condition2       0.000000
BldgType         0.000000
HouseStyle       0.000000
OverallQual      0.000000
OverallCond      0.000000
YearBuilt        0.000000
YearRemodAdd     0.000000
RoofStyle        0.000000
RoofMatl         0.000000
Exterior1st      0.000000
Exterior2nd      0.000000
MasVnrType       0.005479
MasVnrArea       0.005479
ExterQual        0.000000
ExterCond        0.000000
Foundation       0.000000
BsmtQual         0.025342
BsmtCond         0.025342
BsmtExposure     0.026027
BsmtFinType1     0.025342
BsmtFinSF1       0.000000
BsmtFinType2     0.026027
BsmtFinSF2       0.000000
BsmtUnfSF        0.000000
TotalBsmtSF 

Remove columns that contain more than 30% of missing data. After removing those columns, remove the rows that contain at least one observation that is missing.

In [5]:
# Answer below:
housing = housing.drop(columns=['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'])

In [6]:
housing = housing.dropna()

There are some categorical variables that contain numeric data and some that do not. Print the type of each column to first see whether there is an issue with misclassification of column type.

In [7]:
# Answer below:
housing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1094 entries, 0 to 1459
Data columns (total 76 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1094 non-null   int64  
 1   MSSubClass     1094 non-null   int64  
 2   MSZoning       1094 non-null   object 
 3   LotFrontage    1094 non-null   float64
 4   LotArea        1094 non-null   int64  
 5   Street         1094 non-null   object 
 6   LotShape       1094 non-null   object 
 7   LandContour    1094 non-null   object 
 8   Utilities      1094 non-null   object 
 9   LotConfig      1094 non-null   object 
 10  LandSlope      1094 non-null   object 
 11  Neighborhood   1094 non-null   object 
 12  Condition1     1094 non-null   object 
 13  Condition2     1094 non-null   object 
 14  BldgType       1094 non-null   object 
 15  HouseStyle     1094 non-null   object 
 16  OverallQual    1094 non-null   int64  
 17  OverallCond    1094 non-null   int64  
 18  YearBuil

We see that month sold and year sold are not variables that describe a feature of the house. While they do have relevance if we create a model containing a time series element, we will not include them here. Drop these columns. Also, remove the id column

In [8]:
# Answer below
housing = housing.drop(columns=['Id', 'MoSold', 'YrSold'])

Using the information about the column types, identify all the variables that will be converted into dummy variables. Include at least one numeric variable that you think should be converted as well.

In [9]:
housing.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'OpenPorchSF'

In [10]:
# Answer below:
dummy_cols = ['MSZoning','Street',
            'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
            'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
            'OverallQual', 'OverallCond',        
           'RoofStyle',
           'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
           'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
           'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
           'CentralAir', 'Electrical', 'KitchenQual',
           'Functional', 'GarageType', 'GarageFinish',
           'GarageQual', 'GarageCond', 'PavedDrive',
           'SaleType', 'SaleCondition']

Convert the columns you selected above into dummy variables

In [11]:
# Answer below:
dummy_df =  pd.get_dummies(housing[dummy_cols], drop_first=True)
dummy_df

,OverallQual,OverallCond,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,...,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,7,5,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1,6,8,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
2,7,5,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,7,5,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,8,5,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,5,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1456,6,6,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1457,7,9,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0
1458,5,6,0,0,1,0,1,0,0,1,...,0,0,0,0,1,0,0,0,1,0


Split the data into train and test with 20% of data in test.

In [12]:
# Answer below
X = housing.drop(columns=dummy_cols)
X = X.drop(columns=['SalePrice'])
X = pd.concat([X, dummy_df], axis=1)

y = housing['SalePrice']

In [13]:
y

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 1094, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=66)

Create a model with 5 layers. The first layer should be a dense layer that takes in the input, the last layer should be of size 1. You determine the remaining layer sizes.

Use a linear activation for the output layer.

In [21]:
# Answer below
model = Sequential()

# first dense layer
model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
# second dense layer
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))

# output dense layer
model.add(Dense(1, activation='linear')) 

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                2200      
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 2,541
Trainable params: 2,541
Non-trainable params: 0
_________________________________________________________________


Compile the model with the RMSprop optimizer and mean square error loss. Use the MSE as a metric. Set batch size to 100 and epochs to 200. Fit the model and report the results.

In [23]:
# Answer below:
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mse'])

In [24]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=100, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 15ms/step - loss: 41536688128.0000 - mse: 41536688128.0000 - val_loss: 42877804544.0000 - val_mse: 42877804544.0000
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 41468387328.0000 - mse: 41468387328.0000 - val_loss: 42796306432.0000 - val_mse: 42796306432.0000
Epoch 3/200
9/9 [==============================] - 0s 3ms/step - loss: 41380442112.0000 - mse: 41380442112.0000 - val_loss: 42687275008.0000 - val_mse: 42687270912.0000
Epoch 4/200
9/9 [==============================] - 0s 3ms/step - loss: 41264836608.0000 - mse: 41264836608.0000 - val_loss: 42544328704.0000 - val_mse: 42544332800.0000
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 41114869760.0000 - mse: 41114869760.0000 - val_loss: 42364882944.0000 - val_mse: 42364882944.0000
Epoch 6/200
9/9 [==============================] - 0s 3ms/step - loss: 40927350784.0000 - mse: 40927350784.0000 - val_loss: 42141966336.0000 - val_mse: 42

9/9 [==============================] - 0s 3ms/step - loss: 6397911552.0000 - mse: 6397911552.0000 - val_loss: 6000110592.0000 - val_mse: 6000110592.0000
Epoch 50/200
9/9 [==============================] - 0s 3ms/step - loss: 6337938432.0000 - mse: 6337938432.0000 - val_loss: 5967687680.0000 - val_mse: 5967687680.0000
Epoch 51/200
9/9 [==============================] - 0s 3ms/step - loss: 6265673728.0000 - mse: 6265673728.0000 - val_loss: 5930362880.0000 - val_mse: 5930362880.0000
Epoch 52/200
9/9 [==============================] - 0s 3ms/step - loss: 6184472576.0000 - mse: 6184472576.0000 - val_loss: 5900876800.0000 - val_mse: 5900876800.0000
Epoch 53/200
9/9 [==============================] - 0s 3ms/step - loss: 6102980608.0000 - mse: 6102980608.0000 - val_loss: 5867170304.0000 - val_mse: 5867170304.0000
Epoch 54/200
9/9 [==============================] - 0s 3ms/step - loss: 6052599296.0000 - mse: 6052599296.0000 - val_loss: 5825554432.0000 - val_mse: 5825554432.0000
Epoch 55/200
9/9 

9/9 [==============================] - 0s 3ms/step - loss: 4056811008.0000 - mse: 4056811008.0000 - val_loss: 4832359424.0000 - val_mse: 4832359424.0000
Epoch 99/200
9/9 [==============================] - 0s 3ms/step - loss: 4009458688.0000 - mse: 4009458688.0000 - val_loss: 4851256832.0000 - val_mse: 4851256832.0000
Epoch 100/200
9/9 [==============================] - 0s 3ms/step - loss: 3998913536.0000 - mse: 3998913536.0000 - val_loss: 4810456576.0000 - val_mse: 4810456576.0000
Epoch 101/200
9/9 [==============================] - 0s 3ms/step - loss: 3971435264.0000 - mse: 3971435264.0000 - val_loss: 4793885184.0000 - val_mse: 4793885184.0000
Epoch 102/200
9/9 [==============================] - 0s 3ms/step - loss: 3922451968.0000 - mse: 3922451968.0000 - val_loss: 4783116288.0000 - val_mse: 4783116288.0000
Epoch 103/200
9/9 [==============================] - 0s 3ms/step - loss: 3899457280.0000 - mse: 3899457280.0000 - val_loss: 4764075008.0000 - val_mse: 4764075008.0000
Epoch 104/200

9/9 [==============================] - 0s 3ms/step - loss: 2808439552.0000 - mse: 2808439552.0000 - val_loss: 4225131264.0000 - val_mse: 4225131264.0000
Epoch 148/200
9/9 [==============================] - 0s 3ms/step - loss: 2780103936.0000 - mse: 2780103936.0000 - val_loss: 4198801408.0000 - val_mse: 4198801408.0000
Epoch 149/200
9/9 [==============================] - 0s 3ms/step - loss: 2746010880.0000 - mse: 2746010880.0000 - val_loss: 4157455104.0000 - val_mse: 4157455104.0000
Epoch 150/200
9/9 [==============================] - 0s 3ms/step - loss: 2747753472.0000 - mse: 2747753472.0000 - val_loss: 4196568320.0000 - val_mse: 4196568320.0000
Epoch 151/200
9/9 [==============================] - 0s 3ms/step - loss: 2711614720.0000 - mse: 2711614720.0000 - val_loss: 4170945024.0000 - val_mse: 4170945024.0000
Epoch 152/200
9/9 [==============================] - 0s 3ms/step - loss: 2687500800.0000 - mse: 2687500800.0000 - val_loss: 4156200448.0000 - val_mse: 4156200448.0000
Epoch 153/20

9/9 [==============================] - 0s 3ms/step - loss: 2073298944.0000 - mse: 2073298944.0000 - val_loss: 4125538816.0000 - val_mse: 4125539072.0000
Epoch 197/200
9/9 [==============================] - 0s 3ms/step - loss: 2077141632.0000 - mse: 2077141248.0000 - val_loss: 4028809728.0000 - val_mse: 4028809728.0000
Epoch 198/200
9/9 [==============================] - 0s 3ms/step - loss: 2059600384.0000 - mse: 2059600384.0000 - val_loss: 4037110016.0000 - val_mse: 4037110016.0000
Epoch 199/200
9/9 [==============================] - 0s 3ms/step - loss: 2049618816.0000 - mse: 2049618688.0000 - val_loss: 3979313408.0000 - val_mse: 3979313408.0000
Epoch 200/200
9/9 [==============================] - 0s 3ms/step - loss: 2037496576.0000 - mse: 2037496704.0000 - val_loss: 4046970624.0000 - val_mse: 4046970624.0000


Next, do the same but with mean absolute error loss. Use both MSE and MAE as metrics. Compare the results.

In [25]:
# Answer below:
model.compile(optimizer='rmsprop', loss='mean_absolute_error', metrics=['mse', 'mae'])

In [26]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=100, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 14ms/step - loss: 29980.2520 - mse: 2060468608.0000 - mae: 29980.2520 - val_loss: 32625.1270 - val_mse: 3984685824.0000 - val_mae: 32625.1270
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 29803.9648 - mse: 2053948928.0000 - mae: 29803.9648 - val_loss: 32536.8340 - val_mse: 3959368704.0000 - val_mae: 32536.8340
Epoch 3/200
9/9 [==============================] - 0s 3ms/step - loss: 29706.8574 - mse: 2053415168.0000 - mae: 29706.8574 - val_loss: 32438.2520 - val_mse: 3966093568.0000 - val_mae: 32438.2520
Epoch 4/200
9/9 [==============================] - 0s 3ms/step - loss: 29638.1934 - mse: 2038995840.0000 - mae: 29638.1934 - val_loss: 32352.9570 - val_mse: 3978127104.0000 - val_mae: 32352.9570
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 29573.4062 - mse: 2040625920.0000 - mae: 29573.4062 - val_loss: 32297.7344 - val_mse: 3998764288.0000 - val_mae: 32297.7344
Epoch 6/200
9/9 [=======

Epoch 43/200
9/9 [==============================] - 0s 3ms/step - loss: 28201.9062 - mse: 1852209792.0000 - mae: 28201.9062 - val_loss: 31101.8613 - val_mse: 4261448448.0000 - val_mae: 31101.8613
Epoch 44/200
9/9 [==============================] - 0s 3ms/step - loss: 28105.3027 - mse: 1844746880.0000 - mae: 28105.3027 - val_loss: 31132.8672 - val_mse: 4235287296.0000 - val_mae: 31132.8672
Epoch 45/200
9/9 [==============================] - 0s 3ms/step - loss: 28153.2910 - mse: 1861120256.0000 - mae: 28153.2910 - val_loss: 31087.7891 - val_mse: 4308643328.0000 - val_mae: 31087.7891
Epoch 46/200
9/9 [==============================] - 0s 3ms/step - loss: 28074.6895 - mse: 1832032128.0000 - mae: 28074.6895 - val_loss: 31140.1797 - val_mse: 4229778944.0000 - val_mae: 31140.1797
Epoch 47/200
9/9 [==============================] - 0s 3ms/step - loss: 28065.1836 - mse: 1838765184.0000 - mae: 28065.1836 - val_loss: 31090.5352 - val_mse: 4257810176.0000 - val_mae: 31090.5352
Epoch 48/200
9/9 [==

Epoch 85/200
9/9 [==============================] - 0s 3ms/step - loss: 27817.4336 - mse: 1819808256.0000 - mae: 27817.4336 - val_loss: 30611.5684 - val_mse: 4425522176.0000 - val_mae: 30611.5684
Epoch 86/200
9/9 [==============================] - 0s 3ms/step - loss: 27707.3594 - mse: 1800754560.0000 - mae: 27707.3594 - val_loss: 30709.8457 - val_mse: 4374119936.0000 - val_mae: 30709.8457
Epoch 87/200
9/9 [==============================] - 0s 3ms/step - loss: 27667.7734 - mse: 1799493120.0000 - mae: 27667.7734 - val_loss: 30694.0801 - val_mse: 4379306496.0000 - val_mae: 30694.0801
Epoch 88/200
9/9 [==============================] - 0s 3ms/step - loss: 27651.7988 - mse: 1798628224.0000 - mae: 27651.7988 - val_loss: 30594.5996 - val_mse: 4459112960.0000 - val_mae: 30594.5996
Epoch 89/200
9/9 [==============================] - 0s 3ms/step - loss: 27640.3086 - mse: 1806903808.0000 - mae: 27640.3086 - val_loss: 30610.3203 - val_mse: 4465577472.0000 - val_mae: 30610.3203
Epoch 90/200
9/9 [==

Epoch 127/200
9/9 [==============================] - 0s 2ms/step - loss: 27467.7715 - mse: 1784024448.0000 - mae: 27467.7715 - val_loss: 30390.4785 - val_mse: 4503810048.0000 - val_mae: 30390.4785
Epoch 128/200
9/9 [==============================] - 0s 3ms/step - loss: 27442.1641 - mse: 1790171392.0000 - mae: 27442.1641 - val_loss: 30400.1348 - val_mse: 4502003712.0000 - val_mae: 30400.1348
Epoch 129/200
9/9 [==============================] - 0s 3ms/step - loss: 27362.6953 - mse: 1787307264.0000 - mae: 27362.6953 - val_loss: 30663.2734 - val_mse: 4362453504.0000 - val_mae: 30663.2734
Epoch 130/200
9/9 [==============================] - 0s 3ms/step - loss: 27460.0645 - mse: 1794018560.0000 - mae: 27460.0645 - val_loss: 30451.6270 - val_mse: 4416010752.0000 - val_mae: 30451.6270
Epoch 131/200
9/9 [==============================] - 0s 3ms/step - loss: 27395.5156 - mse: 1789259392.0000 - mae: 27395.5156 - val_loss: 30367.0664 - val_mse: 4531778048.0000 - val_mae: 30367.0664
Epoch 132/200
9

9/9 [==============================] - 0s 3ms/step - loss: 27199.0625 - mse: 1765722496.0000 - mae: 27199.0625 - val_loss: 30366.6836 - val_mse: 4410489344.0000 - val_mae: 30366.6836
Epoch 169/200
9/9 [==============================] - 0s 3ms/step - loss: 27267.5586 - mse: 1781710336.0000 - mae: 27267.5586 - val_loss: 30480.0430 - val_mse: 4377651200.0000 - val_mae: 30480.0430
Epoch 170/200
9/9 [==============================] - 0s 3ms/step - loss: 27231.1113 - mse: 1761759616.0000 - mae: 27231.1113 - val_loss: 30418.3926 - val_mse: 4383398400.0000 - val_mae: 30418.3926
Epoch 171/200
9/9 [==============================] - 0s 3ms/step - loss: 27361.0352 - mse: 1804435712.0000 - mae: 27361.0352 - val_loss: 30390.6836 - val_mse: 4410821632.0000 - val_mae: 30390.6836
Epoch 172/200
9/9 [==============================] - 0s 3ms/step - loss: 27174.3184 - mse: 1773741184.0000 - mae: 27174.3184 - val_loss: 30279.1328 - val_mse: 4489417728.0000 - val_mae: 30279.1328
Epoch 173/200
9/9 [==========

Finally, try your model using mean squared logarithmic error. Compare the results.

In [29]:
# Answer below:
model.compile(optimizer='rmsprop', loss='mean_squared_logarithmic_error', metrics=['mse', 'mae'])

In [30]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=100, epochs=200)

Epoch 1/200
9/9 [==============================] - 0s 15ms/step - loss: 0.0410 - mse: 1791410304.0000 - mae: 27246.4336 - val_loss: 0.0541 - val_mse: 4379306496.0000 - val_mae: 30322.8027
Epoch 2/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0403 - mse: 1768697856.0000 - mae: 27260.5469 - val_loss: 0.0536 - val_mse: 4335779840.0000 - val_mae: 30562.4023
Epoch 3/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0404 - mse: 1787996544.0000 - mae: 27397.1465 - val_loss: 0.0537 - val_mse: 4346420736.0000 - val_mae: 30427.0234
Epoch 4/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0401 - mse: 1792217088.0000 - mae: 27332.8477 - val_loss: 0.0538 - val_mse: 4360392192.0000 - val_mae: 30330.2578
Epoch 5/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0401 - mse: 1803784832.0000 - mae: 27469.3301 - val_loss: 0.0542 - val_mse: 4393226240.0000 - val_mae: 30159.3613
Epoch 6/200
9/9 [==============================] - 0s 3ms/step -

9/9 [==============================] - 0s 3ms/step - loss: 0.0389 - mse: 1806463872.0000 - mae: 27314.5117 - val_loss: 0.0525 - val_mse: 4221740800.0000 - val_mae: 30573.2793
Epoch 44/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0389 - mse: 1837430400.0000 - mae: 27438.9902 - val_loss: 0.0529 - val_mse: 4249434368.0000 - val_mae: 30199.4883
Epoch 45/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0388 - mse: 1809481216.0000 - mae: 27411.1445 - val_loss: 0.0525 - val_mse: 4220788736.0000 - val_mae: 30509.3652
Epoch 46/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0387 - mse: 1805855616.0000 - mae: 27372.2363 - val_loss: 0.0525 - val_mse: 4219725056.0000 - val_mae: 30458.7344
Epoch 47/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0388 - mse: 1809226624.0000 - mae: 27331.8008 - val_loss: 0.0524 - val_mse: 4213997824.0000 - val_mae: 30556.8418
Epoch 48/200
9/9 [==============================] - 0s 3ms/step - loss: 0

Epoch 87/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0381 - mse: 1755321600.0000 - mae: 26901.8770 - val_loss: 0.0519 - val_mse: 4198160640.0000 - val_mae: 30745.9473
Epoch 88/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0382 - mse: 1783572224.0000 - mae: 27064.1094 - val_loss: 0.0519 - val_mse: 4204001280.0000 - val_mae: 30518.3672
Epoch 89/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0381 - mse: 1780754048.0000 - mae: 27134.3047 - val_loss: 0.0525 - val_mse: 4261132544.0000 - val_mae: 29785.1914
Epoch 90/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0382 - mse: 1768907520.0000 - mae: 27078.6523 - val_loss: 0.0520 - val_mse: 4221358336.0000 - val_mae: 30192.6934
Epoch 91/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0381 - mse: 1784383360.0000 - mae: 27131.3711 - val_loss: 0.0533 - val_mse: 4318359552.0000 - val_mae: 29527.3398
Epoch 92/200
9/9 [==============================] - 0s 3ms/s

9/9 [==============================] - 0s 3ms/step - loss: 0.0376 - mse: 1733247232.0000 - mae: 26583.4141 - val_loss: 0.0518 - val_mse: 4245755136.0000 - val_mae: 30196.9043
Epoch 131/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0376 - mse: 1749572736.0000 - mae: 26780.1719 - val_loss: 0.0517 - val_mse: 4239684608.0000 - val_mae: 30216.7539
Epoch 132/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0377 - mse: 1764921472.0000 - mae: 26868.9531 - val_loss: 0.0523 - val_mse: 4296901120.0000 - val_mae: 29606.0098
Epoch 133/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0375 - mse: 1749825536.0000 - mae: 26671.7070 - val_loss: 0.0518 - val_mse: 4249897728.0000 - val_mae: 30046.5000
Epoch 134/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0375 - mse: 1753156352.0000 - mae: 26716.6406 - val_loss: 0.0522 - val_mse: 4290007552.0000 - val_mae: 29608.7559
Epoch 135/200
9/9 [==============================] - 0s 3ms/step - lo

9/9 [==============================] - 0s 3ms/step - loss: 0.0371 - mse: 1701939072.0000 - mae: 26309.1094 - val_loss: 0.0517 - val_mse: 4245410816.0000 - val_mae: 30481.9434
Epoch 174/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0373 - mse: 1745354112.0000 - mae: 26553.9141 - val_loss: 0.0516 - val_mse: 4244323840.0000 - val_mae: 30215.3027
Epoch 175/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0372 - mse: 1744981504.0000 - mae: 26560.1055 - val_loss: 0.0519 - val_mse: 4292624640.0000 - val_mae: 29468.9863
Epoch 176/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0371 - mse: 1726842624.0000 - mae: 26459.2871 - val_loss: 0.0518 - val_mse: 4281220608.0000 - val_mae: 29494.1699
Epoch 177/200
9/9 [==============================] - 0s 3ms/step - loss: 0.0370 - mse: 1730391296.0000 - mae: 26426.6426 - val_loss: 0.0516 - val_mse: 4255905792.0000 - val_mae: 30059.4961
Epoch 178/200
9/9 [==============================] - 0s 3ms/step - lo